# AutoGen代理在生产环境中的可观测性与评估

在本教程中，我们将学习如何使用[Langfuse](https://langfuse.com) **监控[Autogen代理](https://github.com/microsoft/autogen)的内部步骤（追踪）**以及**评估其性能**。

本指南涵盖团队用于快速且可靠地将代理投入生产的**在线**和**离线**评估指标。

**为什么AI代理评估很重要：**
- 在任务失败或产生次优结果时进行问题调试
- 实时监控成本和性能
- 通过持续反馈提高可靠性和安全性


## 第一步：设置环境变量

通过注册 [Langfuse Cloud](https://cloud.langfuse.com/) 或 [自托管 Langfuse](https://langfuse.com/self-hosting) 获取您的 Langfuse API 密钥。

_**注意:** 自托管用户可以使用 [Terraform 模块](https://langfuse.com/self-hosting/azure) 在 Azure 上部署 Langfuse。或者，您可以使用 [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm) 在 Kubernetes 上部署 Langfuse。_


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

设置环境变量后，我们现在可以初始化 Langfuse 客户端。`get_client()` 使用环境变量中提供的凭据初始化 Langfuse 客户端。


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## 第2步：初始化 OpenLit 仪器

现在，我们初始化 [OpenLit](https://github.com/openlit/openlit) 仪器。OpenLit 会自动捕获 AutoGen 操作，并将 OpenTelemetry (OTel) 的跨度导出到 Langfuse。


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## 第三步：运行你的代理

现在我们设置一个多轮代理来测试我们的仪器化。


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### 跟踪结构

Langfuse 记录了一个 **跟踪**，其中包含 **跨度**，表示代理逻辑的每一步。在这里，跟踪包括整体代理运行以及以下子跨度：
- 餐饮规划代理
- 营养师代理

您可以检查这些内容，以准确了解时间的分配情况、使用了多少令牌等：

![Langfuse 中的跟踪树](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[跟踪链接](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## 在线评估

在线评估是指在实际生产环境中对代理进行评估，也就是说，在实际使用过程中进行评估。这包括持续监控代理在真实用户交互中的表现，并分析结果。

### 生产环境中常见的监控指标

1. **成本** — 通过工具记录令牌的使用情况，并通过为每个令牌分配价格，将其转换为大致的成本。
2. **延迟** — 观察完成每个步骤或整个运行所需的时间。
3. **用户反馈** — 用户可以提供直接反馈（点赞/点踩），以帮助优化或纠正代理。
4. **LLM 作为评判者** — 使用一个独立的 LLM 来近乎实时地评估代理的输出（例如，检查是否存在有害内容或输出是否正确）。

以下是这些指标的示例。


#### 1. 成本

以下是一个截图，展示了 `gpt-4o-mini` 调用的使用情况。这有助于识别高成本步骤并优化您的代理。

![成本](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[链接到追踪](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. 延迟

我们还可以查看完成每个步骤所花费的时间。在下面的示例中，整个运行大约耗时3秒，可以按步骤分解。这有助于识别瓶颈并优化您的代理。

![延迟](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[链接到追踪](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. 用户反馈

如果你的代理嵌入在用户界面中，你可以记录直接的用户反馈（例如在聊天界面中的点赞/点踩）。


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

用户反馈随后会被记录在 Langfuse 中：

![用户反馈正在被记录在 Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. 自动化 LLM 作为评审的评分

LLM 作为评审是另一种自动评估代理输出的方法。您可以设置一个单独的 LLM 调用来评估输出的正确性、毒性、风格或任何您关心的标准。

**工作流程**：
1. 您定义一个**评估模板**，例如：“检查文本是否有毒性。”
2. 您设置一个模型作为评审模型；在此情况下，通过 Azure 查询 `gpt-4o-mini`。
3. 每次您的代理生成输出时，您将该输出与模板一起传递给“评审”LLM。
4. 评审 LLM 会回复一个评分或标签，您将其记录到您的可观察性工具中。

Langfuse 示例：

![LLM 作为评审的评估器](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

您可以看到这个示例的答案被评判为“无害”。

![LLM-as-a-Judge 评估分数](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. 可观测性指标概览

所有这些指标都可以在仪表板中一起可视化。这使您能够快速了解代理在多个会话中的表现，并帮助您随着时间推移跟踪质量指标。

![可观测性指标概览](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## 离线评估

在线评估对于实时反馈至关重要，但你也需要**离线评估**——在开发之前或过程中进行系统性检查。这有助于在将更改投入生产之前保持质量和可靠性。


### 数据集评估

在离线评估中，通常需要：
1. 准备一个基准数据集（包含提示和预期输出的配对）
2. 在该数据集上运行你的代理
3. 将输出与预期结果进行比较，或者使用额外的评分机制

下面，我们使用 [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) 来演示这一方法，该数据集包含问题和预期答案。


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

接下来，我们在 Langfuse 中创建一个数据集实体以跟踪运行。然后，我们将数据集中的每个项目添加到系统中。


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![数据集项在 Langfuse 中](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### 在数据集上运行代理

首先，我们创建一个简单的 Autogen 代理，它使用 Azure OpenAI 模型回答问题。


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

然后，我们定义一个辅助函数 `my_agent()`。


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

最后，我们遍历每个数据集项，运行代理，并将追踪链接到数据集项。如果需要，我们还可以附加一个快速评估分数。


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

您可以使用不同的代理配置重复此过程，例如：  
- 模型（gpt-4o-mini、gpt-4.1 等）  
- 提示词  
- 工具（搜索 vs. 无搜索）  
- 代理复杂度（多代理 vs. 单代理）  

然后在 Langfuse 中将它们进行并排比较。在这个例子中，我对 25 个数据集问题运行了代理 3 次。每次运行，我使用了不同的 Azure OpenAI 模型。可以看到，使用更大的模型时，正确回答问题的数量有所提高（符合预期）。`correct_answer` 分数是由 [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) 创建的，该评估器被设置为根据数据集中提供的示例答案判断问题的正确性。  

![数据集运行概览](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)  
![数据集运行比较](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)  



---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于重要信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
